In [ ]:
import pyrebase
import json
import pandas as pd

with open('../.streamlit/secrets.txt') as f:
    lines = f.read()
config = json.loads(lines)

firebase = pyrebase.initialize_app(config)

In [ ]:
auth = firebase.auth()
db = firebase.database()
storage = firebase.storage()

In [ ]:
user = auth.create_user_with_email_and_password("sebastien.mauroo@patrick.com","123456")

In [ ]:
try:
    signin = auth.sign_in_with_email_and_password("sebastien.mauroo@outlook.com","1234567")
except:
    signin ={"login failed"}

In [ ]:
signin

In [ ]:
firebase.auth().send_email_verification(signin['idToken'])

In [ ]:
firebase.auth().send_password_reset_email("sebastien.mauroo@outlook.com")

In [ ]:
auth.get_account_info(signin['idToken'])

In [ ]:
data = {"name": "Mortimer 'Morty' Smith"}
db.child("test").set(data)

In [123]:
dt = pd.read_csv("testdata_fluvius.csv",delimiter=';',decimal=',')

In [125]:
dt

,Van Datum,Van Tijdstip,Tot Datum,Tot Tijdstip,EAN,Meter,Metertype,Register,Volume,Eenheid,Validatiestatus,start_time,end_time
0,14-10-2022,00:00:00,14-10-2022,00:15:00,"=""541448860008410850""",1SAG3100010760,Digitale Meter,Afname Nacht,NaN,kWh,Geen verbruik,2022-10-14 00:00:00,2022-10-14 00:15:00
1,14-10-2022,00:00:00,14-10-2022,00:15:00,"=""541448860008410850""",1SAG3100010760,Digitale Meter,Injectie Nacht,NaN,kWh,Geen verbruik,2022-10-14 00:00:00,2022-10-14 00:15:00
2,14-10-2022,00:15:00,14-10-2022,00:30:00,"=""541448860008410850""",1SAG3100010760,Digitale Meter,Afname Nacht,0.093,kWh,Gevalideerd,2022-10-14 00:15:00,2022-10-14 00:30:00
3,14-10-2022,00:15:00,14-10-2022,00:30:00,"=""541448860008410850""",1SAG3100010760,Digitale Meter,Injectie Nacht,0.000,kWh,Gevalideerd,2022-10-14 00:15:00,2022-10-14 00:30:00
4,14-10-2022,00:30:00,14-10-2022,00:45:00,"=""541448860008410850""",1SAG3100010760,Digitale Meter,Afname Nacht,0.120,kWh,Gevalideerd,2022-10-14 00:30:00,2022-10-14 00:45:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
26115,26-02-2023,23:15:00,26-02-2023,23:30:00,"=""541448860008410850""",1SAG3100010760,Digitale Meter,Injectie Nacht,0.000,kWh,Gevalideerd,2023-02-26 23:15:00,2023-02-26 23:30:00
26116,26-02-2023,23:30:00,26-02-2023,23:45:00,"=""541448860008410850""",1SAG3100010760,Digitale Meter,Afname Nacht,0.125,kWh,Gevalideerd,2023-02-26 23:30:00,2023-02-26 23:45:00
26117,26-02-2023,23:30:00,26-02-2023,23:45:00,"=""541448860008410850""",1SAG3100010760,Digitale Meter,Injectie Nacht,0.000,kWh,Gevalideerd,2023-02-26 23:30:00,2023-02-26 23:45:00
26118,26-02-2023,23:45:00,27-02-2023,00:00:00,"=""541448860008410850""",1SAG3100010760,Digitale Meter,Afname Nacht,0.126,kWh,Gevalideerd,2023-02-26 23:45:00,2023-02-27 00:00:00


In [126]:
###get start period
dt['start_time'] = pd.to_datetime(dt.iloc[:,0] + " " + dt.iloc[:,1],format='%d-%m-%Y %H:%M:%S')

###get end period
dt['end_time'] = pd.to_datetime(dt.iloc[:,2] + " " + dt.iloc[:,3],format='%d-%m-%Y %H:%M:%S')

In [127]:
#get repeated parameters
EAN_code = dt["EAN"].iloc[0].replace('=','').replace('"','')
Meter_code = dt["Meter"].iloc[0]
Meter_type = dt["Metertype"].iloc[0]
Power_unit = dt["Eenheid"].iloc[0]
Time_unit = (dt["end_time"].iloc[0] - dt["start_time"].iloc[0]).seconds/60 ##15 bij kwartier waarden, 60 bij uurwaarden
Data_period = (dt["start_time"].iloc[-1] - dt["start_time"].iloc[0]).round('d').days

In [105]:
dt = dt.dropna()[['start_time','end_time','Volume','Register']]

In [121]:
#########AANPASSEN NAAR INJECTIE

######injection analysis

#Estimated solar capacity
var1 = dt[dt['Register'].str.contains('Afname')]['Volume'].nlargest(3) ###largest 3 injectinos
Estimated_generation_capacity = var1.mean()*60/Time_unit ###60/Time_unit converts kWh towards kW

#Has solar panels
Has_solar_panels = True if Estimated_generation_capacity >= 0.6 else False  ###2 solar panels = +-600W

In [122]:
dt[dt['Register'].str.contains('Afname')]['Volume'].nlargest(3) ###largest 3 injectinos

18148    1.212
14506    1.209
4922     1.173
Name: Volume, dtype: float64

In [107]:
dt

,start_time,end_time,Volume,Register
2,2022-10-14 00:15:00,2022-10-14 00:30:00,0.093,Afname Nacht
3,2022-10-14 00:15:00,2022-10-14 00:30:00,0.000,Injectie Nacht
4,2022-10-14 00:30:00,2022-10-14 00:45:00,0.120,Afname Nacht
5,2022-10-14 00:30:00,2022-10-14 00:45:00,0.000,Injectie Nacht
6,2022-10-14 00:45:00,2022-10-14 01:00:00,0.113,Afname Nacht
...,...,...,...,...
26115,2023-02-26 23:15:00,2023-02-26 23:30:00,0.000,Injectie Nacht
26116,2023-02-26 23:30:00,2023-02-26 23:45:00,0.125,Afname Nacht
26117,2023-02-26 23:30:00,2023-02-26 23:45:00,0.000,Injectie Nacht
26118,2023-02-26 23:45:00,2023-02-27 00:00:00,0.126,Afname Nacht


In [119]:
graphtable = dt.pivot_table(index='end_time', columns='Register', values='Volume',aggfunc='mean').fillna(0)
graphtable['Afname'] = graphtable['Afname Dag'] + graphtable['Afname Nacht']
graphtable['Injectie'] = graphtable['Injectie Dag'] + graphtable['Injectie Nacht']
graphtable[['Afname','Injectie']]

In [120]:
graphtable[['Afname','Injectie']]

Register,Afname,Injectie
end_time,,
2022-01-11 00:00:00,0.106,0.0
2022-01-11 00:15:00,0.133,0.0
2022-01-11 00:30:00,0.117,0.0
2022-01-11 00:45:00,0.093,0.0
2022-01-11 01:00:00,0.096,0.0
...,...,...
2023-12-02 22:45:00,0.159,0.0
2023-12-02 23:00:00,0.128,0.0
2023-12-02 23:15:00,0.149,0.0
